# **Data Science Capstone**

This notebook will be used for the Coursera Capstone Project 

In [4]:
import pandas as pd
import numpy as np

In [5]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


## Introduction

A friend of mine is moving to New York City to open an Indian restaurant. I have decided to help her choose which neighborhood in NYC she should start her business. She has a few requirements including an area with a high volume of foot traffic which could translate to a more business centered area than residential. She also would not like to be near other Indian restaurants. Lastly, she does not want to work too far from where she lives. She will be moving to Brooklyn and would want to be within a 500 meter radius from her home.

## Interest

In this  particular case, the only stake holder would be my friend who is opening the restaurant. However, this analysis can prove to be useful to anyone wanting information about particular areas of New York City.

## Data Used

* New York Cities & Boroughs dataset - geo.nyu.edu from previous lab
* Restaurants - Foursquare
* Plazas, Art Galleries, Parks (Places with high foot traffic) - Foursquare

## Methodology

1. **Data Cleaning** - Data downloaded from the above New York City website and Foursquare will then be cleaned and combined into one table.
2. **Feature Selection** - After cleaning the data, I will analyze each neighborhood and reform it to be around Brooklyn exactly where my friend will be living.
3. **Exploratory Data Analysis** - I will take the mean frequency of each venue category in the neighborhood and display them in a new dataframe. Then, I will create a new dataframe from this last with the neighborhoods and there top 10 most popular venues. I will then use a k mean cluster to plot these findings on a map of NYC.
4. **Examine Clusters** - Once I have created the cluster neighborhoods on the map, I will examine each cluster to see which ones satisfy my friends needs best.

### 1. Data Cleaning

#### First download all the necessary packages 

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


#### Download and explore New York Dataset

In [3]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [4]:
# Transfer it from json to a dictionary
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

#### From the previous lab, we know all the relevant information is in the features key so we will extract that and input it into our neighborhoods dictionary.

In [5]:
neighborhoods_data = newyork_data['features']
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

#### Transform the data into a pandas dataframe

In [6]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [7]:
# loop through the data and transfer it to our newly created dataframe
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

#### Now, examine the results of the dataframe

In [8]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


### 2. Feature Selection

#### Use geopy library to get the latitude and longitude values of where my friend will live in New York. (For the purpose of this project, I will use the address of the hotel, The Brooklyn, in Brooklyn)

In [11]:
address = '1199 Atlantic Ave, Brooklyn, NY '

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of The Brooklyn are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of The Brooklyn are 40.678781400000005, -73.95118321451463.


#### Since my friend will be staying in Brooklyn, we can focus on the data in Brooklyn by creating a new dataset with this requirement.

In [13]:
brooklyn_data = neighborhoods[neighborhoods['Borough'] == 'Brooklyn'].reset_index(drop=True)
brooklyn_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Brooklyn,Bay Ridge,40.625801,-74.030621
1,Brooklyn,Bensonhurst,40.611009,-73.995180
2,Brooklyn,Sunset Park,40.645103,-74.010316
3,Brooklyn,Greenpoint,40.730201,-73.954241
4,Brooklyn,Gravesend,40.595260,-73.973471


#### Now let's visualize Brooklyn and all the neighborhoods in it.

In [14]:
# create map of Brooklyn using latitude and longitude values
map_brooklyn = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(brooklyn_data['Latitude'], brooklyn_data['Longitude'], brooklyn_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_brooklyn)  
    
map_brooklyn

### 3. Exploratory Data Analysis

#### Now, we will utilize Foursquare API to explore Brooklyn's neighborhoods and segment them. First, by defining the Foursquare credentials and version.

In [15]:
CLIENT_ID = '4LWLRZEEVDEDRRDTOGDFBWBDSK23WBMDKWNCQYMQAWZ34VOU' # your Foursquare ID
CLIENT_SECRET = 'IWTTWCVIBSXG5EZS5LOEQOC3FNT0D5ZLQM5UTNLQL1S23LSM' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 4LWLRZEEVDEDRRDTOGDFBWBDSK23WBMDKWNCQYMQAWZ34VOU
CLIENT_SECRET:IWTTWCVIBSXG5EZS5LOEQOC3FNT0D5ZLQM5UTNLQL1S23LSM


#### Now, it is time to get the top 100 venues with a radius of 500 meters of The Brooklyn

In [17]:
radius = 500
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)

In [21]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ec74b3b618f43001b8d70f1'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'Bedford-Stuyvesant',
  'headerFullLocation': 'Bedford-Stuyvesant, Brooklyn',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 47,
  'suggestedBounds': {'ne': {'lat': 40.68328140450001,
    'lng': -73.9452605537403},
   'sw': {'lat': 40.6742813955, 'lng': -73.95710587528896}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '51f70290498e7aa5c9f47324',
       'name': 'India House',
       'location': {'address': '586 Nostrand Ave',
        'crossStreet': 'Atlantic Ave',
        'lat': 40.6787082701176,
        'lng': -

#### Let's extract the venue category from the data by borrowing the get category function from the previous lab

In [22]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#### Now, we can clean up the data and transfer it to a dataframe

In [23]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,India House,Indian Restaurant,40.678708,-73.949651
1,David's Brisket House,Sandwich Place,40.679134,-73.949617
2,Ali's T & T Roti,Caribbean Restaurant,40.680497,-73.950719
3,A & A Bake & Doubles,Caribbean Restaurant,40.680807,-73.949525
4,Café Cotton Bean,Coffee Shop,40.676133,-73.950327


In [24]:
# to see how many venues were returned
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

47 venues were returned by Foursquare.


#### Now we can explore the neighborhoods in Brooklyn by again borrowing a function from the previous lab

In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [29]:
# now run the function
brooklyn_venues = getNearbyVenues(names=brooklyn_data['Neighborhood'],
                                   latitudes=brooklyn_data['Latitude'],
                                   longitudes=brooklyn_data['Longitude']
                                  )

Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker Heights
Gerritsen Beach
Marine Park
Clinton Hill
Sea Gate
Downtown
Boerum Hill
Prospect Lefferts Gardens
Ocean Hill
City Line
Bergen Beach
Midwood
Prospect Park South
Georgetown
East Williamsburg
North Side
South Side
Ocean Parkway
Fort Hamilton
Ditmas Park
Wingate
Rugby
Remsen Village
New Lots
Paerdegat Basin
Mill Basin
Fulton Ferry
Vinegar Hill
Weeksville
Broadway Junction
Dumbo
Homecrest
Highland Park
Madison
Erasmus


In [30]:
# Analyze how many unique categories can derive from the data
print('There are {} uniques categories.'.format(len(brooklyn_venues['Venue Category'].unique())))

There are 286 uniques categories.


#### Analyze each neighborhood

In [31]:
# one hot encoding
brooklyn_onehot = pd.get_dummies(brooklyn_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
brooklyn_onehot['Neighborhood'] = brooklyn_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [brooklyn_onehot.columns[-1]] + list(brooklyn_onehot.columns[:-1])
brooklyn_onehot = brooklyn_onehot[fixed_columns]

brooklyn_onehot.head()

,Yoga Studio,Accessories Store,African Restaurant,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,History Museum,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundromat,Laundry Service,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Nightlife Spot,Non-Profit,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Piercing Parlor,Pilates Studio,Pizza Place,Playground,Plaza,Polish Restaurant,Pool,Pool Hall,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Rock Club,Roller Rink,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Tiki Bar,Toy / Game Store,Trail,Turkish Restaurant,Used Bookstor

#### Now we can group rows by neighborhoods and take the mean frequency of each occurence in each category

In [33]:
brooklyn_grouped = brooklyn_onehot.groupby('Neighborhood').mean().reset_index()
brooklyn_grouped

,Neighborhood,Yoga Studio,Accessories Store,African Restaurant,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,History Museum,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundromat,Laundry Service,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Nightlife Spot,Non-Profit,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Piercing Parlor,Pilates Studio,Pizza Place,Playground,Plaza,Polish Restaurant,Pool,Pool Hall,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Rock Club,Roller Rink,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Tiki Bar,Toy / Game Store,Trail,Turkish Restaurant,Used Bookstor

#### Now let's print each neighborhood along with there top 5 venues

In [34]:
num_top_venues = 5

for hood in brooklyn_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = brooklyn_grouped[brooklyn_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bath Beach----
                venue  freq
0  Chinese Restaurant  0.06
1            Pharmacy  0.06
2  Italian Restaurant  0.04
3     Bubble Tea Shop  0.04
4    Sushi Restaurant  0.04


----Bay Ridge----
                 venue  freq
0                  Spa  0.07
1   Italian Restaurant  0.07
2          Pizza Place  0.06
3  American Restaurant  0.03
4     Greek Restaurant  0.03


----Bedford Stuyvesant----
           venue  freq
0    Coffee Shop  0.08
1           Café  0.08
2    Pizza Place  0.08
3            Bar  0.08
4  Deli / Bodega  0.08


----Bensonhurst----
                venue  freq
0       Grocery Store  0.07
1  Italian Restaurant  0.07
2          Donut Shop  0.07
3  Chinese Restaurant  0.07
4    Sushi Restaurant  0.07


----Bergen Beach----
                venue  freq
0     Harbor / Marina  0.33
1      Baseball Field  0.17
2          Playground  0.17
3          Donut Shop  0.17
4  Athletics & Sports  0.17


----Boerum Hill----
               venue  freq
0        Coffee Shop  

#### Now, let's put that data into a pandas dataframe to get a clearer picture

In [35]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### First we created a function to sort it in descending order and now we will create the dataframe grouping the neighborhoods with there top 10 venues

In [36]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = brooklyn_grouped['Neighborhood']

for ind in np.arange(brooklyn_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(brooklyn_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bath Beach,Pharmacy,Chinese Restaurant,Gas Station,Fast Food Restaurant,Bubble Tea Shop,Italian Restaurant,Sushi Restaurant,Shoe Store,Sandwich Place,Peruvian Restaurant
1,Bay Ridge,Spa,Italian Restaurant,Pizza Place,Greek Restaurant,American Restaurant,Grocery Store,Bar,Gym / Fitness Center,Bagel Shop,Sandwich Place
2,Bedford Stuyvesant,Café,Deli / Bodega,Pizza Place,Bar,Coffee Shop,Tiki Bar,New American Restaurant,Gift Shop,Gourmet Shop,Cocktail Bar
3,Bensonhurst,Sushi Restaurant,Grocery Store,Chinese Restaurant,Ice Cream Shop,Donut Shop,Bakery,Italian Restaurant,Noodle House,Sporting Goods Shop,Café
4,Bergen Beach,Harbor / Marina,Donut Shop,Playground,Athletics & Sports,Baseball Field,Women's Store,Factory,Falafel Restaurant,Farm,Farmers Market


### Cluster Neighborhoods

#### Run k-means to cluster the neighborhoods into 7 clusters, I have chosen arbitrarily

In [37]:
# set number of clusters
kclusters = 7

brooklyn_grouped_clustering = brooklyn_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(brooklyn_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([5, 0, 0, 5, 2, 0, 5, 5, 2, 0], dtype=int32)

#### We will create a new dataframe that includes the clusters

In [38]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

brooklyn_merged = brooklyn_data

# add latitude/longitude for each neighborhood
brooklyn_merged = brooklyn_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

brooklyn_merged.head() 

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Brooklyn,Bay Ridge,40.625801,-74.030621,0,Spa,Italian Restaurant,Pizza Place,Greek Restaurant,American Restaurant,Grocery Store,Bar,Gym / Fitness Center,Bagel Shop,Sandwich Place
1,Brooklyn,Bensonhurst,40.611009,-73.995180,5,Sushi Restaurant,Grocery Store,Chinese Restaurant,Ice Cream Shop,Donut Shop,Bakery,Italian Restaurant,Noodle House,Sporting Goods Shop,Café
2,Brooklyn,Sunset Park,40.645103,-74.010316,5,Pizza Place,Mexican Restaurant,Latin American Restaurant,Bank,Bakery,Pharmacy,Mobile Phone Shop,Fried Chicken Joint,Gym,Women's Store
3,Brooklyn,Greenpoint,40.730201,-73.954241,0,Bar,Pizza Place,Coffee Shop,Cocktail Bar,Yoga Studio,Deli / Bodega,Sushi Restaurant,French Restaurant,Grocery Store,Café
4,Brooklyn,Gravesend,40.595260,-73.973471,5,Italian Restaurant,Pizza Place,Bakery,Bus Station,Lounge,Pharmacy,Cosmetics Shop,Donut Shop,Eastern European Restaurant,Chinese Restaurant


#### Now we will visualize the resulting clusters

In [39]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(brooklyn_merged['Latitude'], brooklyn_merged['Longitude'], brooklyn_merged['Neighborhood'], brooklyn_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### 4. Examine Clusters

#### Now we will examine each cluster and determine which would work best for my friend's business location

**Cluster 1**

In [44]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 0, brooklyn_merged.columns[[1] + list(range(7, brooklyn_merged.shape[1]))]]

,Neighborhood,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bay Ridge,Pizza Place,Greek Restaurant,American Restaurant,Grocery Store,Bar,Gym / Fitness Center,Bagel Shop,Sandwich Place
3,Greenpoint,Coffee Shop,Cocktail Bar,Yoga Studio,Deli / Bodega,Sushi Restaurant,French Restaurant,Grocery Store,Café
12,Windsor Terrace,Plaza,Café,Diner,Coffee Shop,Chinese Restaurant,Sushi Restaurant,Salad Place,Bookstore
13,Prospect Heights,Café,Thai Restaurant,Cocktail Bar,Gourmet Shop,Bakery,Wine Bar,Wine Shop,Jewelry Store
15,Williamsburg,Bagel Shop,Yoga Studio,Diner,Breakfast Spot,Burger Joint,Bus Station,Café,Clothing Store
16,Bushwick,Coffee Shop,Pizza Place,Deli / Bodega,Bakery,Discount Store,Thrift / Vintage Store,Vegetarian / Vegan Restaurant,Chinese Restaurant
17,Bedford Stuyvesant,Pizza Place,Bar,Coffee Shop,Tiki Bar,New American Restaurant,Gift Shop,Gourmet Shop,Cocktail Bar
18,Brooklyn Heights,Park,Bakery,Italian Restaurant,Mexican Restaurant,Pizza Place,Gym,American Restaurant,Thai Restaurant
19,Cobble Hill,Bar,Cocktail Bar,Coffee Shop,Yoga Studio,Italian Restaurant,Ice Cream Shop,Deli / Bodega,Middle Eastern Restaurant
20,Carroll Gardens,Pizza Place,Cocktail Bar,Bakery,Wine Shop,Bar,Spa,Gift Shop,French Restaurant


**Cluster 2**

In [43]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 1, brooklyn_merged.columns[[1] + list(range(7, brooklyn_merged.shape[1]))]]

,Neighborhood,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
46,Midwood,Field,Candy Store,Bagel Shop,Convenience Store,Video Game Store,Bakery,Ice Cream Shop,Discount Store


**Cluster 3**

In [45]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 2, brooklyn_merged.columns[[1] + list(range(7, brooklyn_merged.shape[1]))]]

,Neighborhood,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,East Flatbush,Food & Drink Shop,Chinese Restaurant,Park,Hardware Store,Fast Food Restaurant,Supermarket,Moving Target,Caribbean Restaurant
14,Brownsville,Chinese Restaurant,Park,Playground,Fried Chicken Joint,Discount Store,Performing Arts Venue,Spanish Restaurant,Airport Terminal
25,Cypress Hills,Ice Cream Shop,Donut Shop,Chinese Restaurant,Fast Food Restaurant,Dance Studio,Discount Store,Coffee Shop,Bank
27,Starrett City,Donut Shop,Bus Stop,Bus Station,Caribbean Restaurant,Convenience Store,Pizza Place,American Restaurant,Liquor Store
29,Flatlands,Fast Food Restaurant,Fried Chicken Joint,Bar,Lounge,Chinese Restaurant,Gym,Nightclub,Paper / Office Supplies Store
45,Bergen Beach,Playground,Athletics & Sports,Baseball Field,Women's Store,Factory,Falafel Restaurant,Farm,Farmers Market
55,Wingate,Playground,Bus Station,Farmers Market,Bakery,BBQ Joint,Gym / Fitness Center,Donut Shop,Field
57,Remsen Village,Deli / Bodega,Donut Shop,Fried Chicken Joint,Fish Market,Supermarket,Sandwich Place,Salad Place,Gym
59,Paerdegat Basin,Bus Line,Harbor / Marina,Auto Garage,Asian Restaurant,Women's Store,Fast Food Restaurant,Falafel Restaurant,Farm
64,Broadway Junction,Fried Chicken Joint,Caribbean Restaurant,Bakery,Gas Station,Nightclub,Business Service,Sandwich Place,Japanese Restaurant


**Cluster 4**

In [46]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 3, brooklyn_merged.columns[[1] + list(range(7, brooklyn_merged.shape[1]))]]

,Neighborhood,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,Mill Island,Food Stand,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field


**Cluster 5**

In [47]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 4, brooklyn_merged.columns[[1] + list(range(7, brooklyn_merged.shape[1]))]]

,Neighborhood,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
68,Madison,Pizza Place,Italian Restaurant,Candy Store,Restaurant,Spa,Pilates Studio,Dessert Shop,Diner


**Cluster 6**

In [48]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 5, brooklyn_merged.columns[[1] + list(range(7, brooklyn_merged.shape[1]))]]

,Neighborhood,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Bensonhurst,Chinese Restaurant,Ice Cream Shop,Donut Shop,Bakery,Italian Restaurant,Noodle House,Sporting Goods Shop,Café
2,Sunset Park,Latin American Restaurant,Bank,Bakery,Pharmacy,Mobile Phone Shop,Fried Chicken Joint,Gym,Women's Store
4,Gravesend,Bakery,Bus Station,Lounge,Pharmacy,Cosmetics Shop,Donut Shop,Eastern European Restaurant,Chinese Restaurant
5,Brighton Beach,Russian Restaurant,Beach,Sushi Restaurant,Bank,Mobile Phone Shop,Gourmet Shop,Playground,Food & Drink Shop
6,Sheepshead Bay,Sandwich Place,Grocery Store,Pizza Place,Diner,Creperie,Park,Outlet Store,Restaurant
7,Manhattan Terrace,Convenience Store,Ice Cream Shop,Eastern European Restaurant,Mobile Phone Shop,Bus Stop,Steakhouse,Organic Grocery,Coffee Shop
8,Flatbush,Mexican Restaurant,Coffee Shop,Donut Shop,Chinese Restaurant,Lounge,Bank,Bakery,Bagel Shop
9,Crown Heights,Café,Pharmacy,Coffee Shop,Bookstore,Candy Store,Supermarket,Sushi Restaurant,Salon / Barbershop
11,Kensington,Pizza Place,Ice Cream Shop,Sandwich Place,Lingerie Store,Liquor Store,Spa,Department Store,Burger Joint
26,East New York,Asian Restaurant,Pizza Place,Pharmacy,Convenience Store,Caribbean Restaurant,Event Service,Fast Food Restaurant,Music Venue


**Cluster 7**

In [49]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 6, brooklyn_merged.columns[[1] + list(range(7, brooklyn_merged.shape[1]))]]

,Neighborhood,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
39,Sea Gate,Beach,Home Service,Construction & Landscaping,Filipino Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant


### Conclusion

Any neighborhood in Cluster 6 would be a great place for my friend to open up her restaurant since it covers all the requirements she has asked for. By process of elimination , we can tell Clusters 2, 4, 5, and 7 are too small so probably not enough foot traffic. That leaves Clusters 1, 3, and 6. Cluster 6 had banks, clothing stores, parks, and beaches which in turn would mean a lot more foot traffic for her business.